In [ ]:
# — new cell —

import pandas as pd

# load
# current
df1 = pd.read_csv('../../data/processed/rads/rads_22_06_2025.csv')
# past
df2 = pd.read_csv('../../data/processed/rads/rads_28_04_2025.csv')

# sort & rank within each provider + room_type
df1_sorted = (
    df1
    .sort_values(['nid', 'room_type', 'maximumRAD'])
    .assign(room_rank=lambda d: d.groupby(['nid','room_type']).cumcount() + 1)
)
df2_sorted = (
    df2
    .sort_values(['nid', 'room_type', 'maximumRAD'])
    .assign(room_rank=lambda d: d.groupby(['nid','room_type']).cumcount() + 1)
)

# rename the RAD columns for clarity
curr = df1_sorted.rename(columns={'maximumRAD':'maximumRAD_current'})
past = df2_sorted.rename(columns={'maximumRAD':'maximumRAD_past'})

# merge on nid, room_type, and our new room_rank
compare = curr.merge(
    past,
    on=['nid','room_type','room_rank'],
    how='inner'
)

# compute price deltas
compare['rad_price_change']     = compare['maximumRAD_current'] - compare['maximumRAD_past']
compare['rad_price_pct_change'] = compare['rad_price_change'] / compare['maximumRAD_past'] * 100

# export to CSV
out_path = '../../data/processed/rads/rad_price_comparison_ranked__06_vs_28_04_2025.csv'
compare.to_csv(out_path, index=False)

print(f"Saved comparison to {out_path}")

Saved comparison to ../../data/processed/rads/rad_price_comparison_ranked_22_06_vs_28_04_2025.csv
